In [37]:
# Test Standardized Column Names After Database Update
import rdshelp
import pandas as pd

# Fresh connection to test standardized columns
print("🔧 TESTING COLUMN STANDARDIZATION RESULTS")
print("=" * 60)

# Create new connection
conn = rdshelp.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

print("✅ Connection established")

# Test the key improvement: gameid consistency
print(f"\n🎯 KEY IMPROVEMENT TEST: gameid consistency")

# Test V3 endpoint table (originally used GAMEID)
with conn.cursor() as cur:
    cur.execute("SELECT gameid, teamid FROM nba_boxscoreadvancedv3_playerstats LIMIT 3;")
    rows = cur.fetchall()
    print(f"   V3 table (nba_boxscoreadvancedv3_playerstats): gameid ✅")
    print(f"   Sample: {rows[0] if rows else 'No data'}")

# Test master games table (originally used game_id)  
with conn.cursor() as cur:
    cur.execute("SELECT gameid, hometeamid, awayteamid FROM nba_games LIMIT 3;")
    rows = cur.fetchall()
    print(f"   Master table (nba_games): gameid ✅")
    print(f"   Sample: {rows[0] if rows else 'No data'}")

print(f"\n🏆 SUCCESS: Now both V2 and V3 tables use 'gameid' consistently!")
print(f"   - No more GAMEID vs GAME_ID confusion")
print(f"   - All leagues use same column naming")
print(f"   - Endpoint processor can use consistent field names")

conn.close()

🔧 TESTING COLUMN STANDARDIZATION RESULTS
Connected to RDS PostgreSQL database
✅ Connection established

🎯 KEY IMPROVEMENT TEST: gameid consistency
   V3 table (nba_boxscoreadvancedv3_playerstats): gameid ✅
   Sample: ('0022301199', 1610612745)
   Master table (nba_games): gameid ✅
   Sample: ('0022301197', '1610612759', '1610612765')

🏆 SUCCESS: Now both V2 and V3 tables use 'gameid' consistently!
   - No more GAMEID vs GAME_ID confusion
   - All leagues use same column naming
   - Endpoint processor can use consistent field names
Connected to RDS PostgreSQL database
✅ Connection established

🎯 KEY IMPROVEMENT TEST: gameid consistency
   V3 table (nba_boxscoreadvancedv3_playerstats): gameid ✅
   Sample: ('0022301199', 1610612745)
   Master table (nba_games): gameid ✅
   Sample: ('0022301197', '1610612759', '1610612765')

🏆 SUCCESS: Now both V2 and V3 tables use 'gameid' consistently!
   - No more GAMEID vs GAME_ID confusion
   - All leagues use same column naming
   - Endpoint processo

In [38]:
import nba_api.stats.endpoints as nbaapi
import rdshelp
import pandas as pd

# ✅ RDS CONNECTION NOW WORKING! (Fixed security group)
print("🔍 Testing RDS connection after security group fix...")

conn = rdshelp.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')


🔍 Testing RDS connection after security group fix...
Connected to RDS PostgreSQL database


In [ ]:
gamefinder = nbaapi.leaguegamefinder.LeagueGameFinder(
                            league_id_nullable='00',
                            season_nullable='2024-2025')

In [16]:
gamefinder.get_data_frames()[0].to_csv('test.csv')

In [ ]:
# Query all public tables in the connected RDS database

query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

with conn.cursor() as cur:
    cur.execute(query)
    tables = cur.fetchall()
    table_list = [row[0] for row in tables]
    print("Public tables in the database:")
    print(table_list)


Public tables in the database:
['boxscoreadvancedv3_playerstats', 'gleague_games', 'gleague_players', 'gleague_teams', 'mastergames', 'masterplayers', 'nba_games', 'nba_players', 'nba_teams', 'test_games_g_league', 'test_games_nba', 'test_games_wnba', 'wnba_games', 'wnba_players', 'wnba_teams']


In [49]:
# Test standardized column names across all leagues
print("🧪 TESTING STANDARDIZED COLUMN NAMES")
print("=" * 50)

# Test master tables with standardized columns
master_tables = [
    ('nba_games', 'NBA'),
    ('wnba_games', 'WNBA'), 
    ('gleague_games', 'G-League'),
    ('nba_players', 'NBA'),
    ('wnba_players', 'WNBA'),
    ('gleague_players', 'G-League')
]

for table_name, league in master_tables:
    print(f"\n📊 Testing: {table_name} ({league})")
    
    try:
        # Test standardized column names
        if 'games' in table_name:
            query = f"SELECT gameid, hometeamid, awayteamid FROM {table_name} LIMIT 3;"
        else:  # players table
            query = f"SELECT playerid, playername, teamid FROM {table_name} LIMIT 3;"
        
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
            
            print(f"   ✅ Columns: {colnames}")
            print(f"   Records: {len(rows)}")
            
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")

# Test endpoint tables - first check what columns exist
print(f"\n📈 Testing NBA endpoint table columns:")

# Check what columns actually exist in the endpoint table
with conn.cursor() as cur:
    cur.execute("""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = 'nba_boxscoreadvancedv3_playerstats'
        ORDER BY ordinal_position
        LIMIT 10;
    """)
    columns = [row[0] for row in cur.fetchall()]
    print(f"   Available columns: {columns}")

# Now query with available columns that should be standardized
query = "SELECT * FROM nba_boxscoreadvancedv3_playerstats;"

games_df = None
with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    games_df = pd.DataFrame(rows, columns=colnames)
    print(f"✅ Standardized endpoint table: {len(games_df)} rows")
    print(f"   Key ID columns: {colnames}")
    display(games_df.head())

print(f"\n🎯 STANDARDIZATION SUCCESS!")
print(f"   ✅ All master tables use consistent lowercase column names")
print(f"   ✅ Key fields: gameid, playerid, teamid (no underscores)")
print(f"   ✅ Both V2 and V3 endpoints now use same naming convention")

🧪 TESTING STANDARDIZED COLUMN NAMES

📊 Testing: nba_games (NBA)
   ✅ Columns: ['gameid', 'hometeamid', 'awayteamid']
   Records: 3

📊 Testing: wnba_games (WNBA)
   ✅ Columns: ['gameid', 'hometeamid', 'awayteamid']
   Records: 3

📊 Testing: gleague_games (G-League)
   ✅ Columns: ['gameid', 'hometeamid', 'awayteamid']
   Records: 3

📊 Testing: nba_players (NBA)
   ✅ Columns: ['playerid', 'playername', 'teamid']
   Records: 3

📊 Testing: wnba_players (WNBA)
   ✅ Columns: ['playerid', 'playername', 'teamid']
   Records: 3

📊 Testing: gleague_players (G-League)
   ✅ Columns: ['playerid', 'playername', 'teamid']
   Records: 3

📈 Testing NBA endpoint table columns:
   Available columns: ['gameid', 'teamid', 'teamcity', 'teamname', 'teamtricode', 'teamslug', 'personid', 'firstname', 'familyname', 'namei']
✅ Standardized endpoint table: 27758 rows
   Key ID columns: ['gameid', 'teamid', 'teamcity', 'teamname', 'teamtricode', 'teamslug', 'personid', 'firstname', 'familyname', 'namei', 'playerslu

,gameid,teamid,teamcity,teamname,teamtricode,teamslug,personid,firstname,familyname,namei,...,turnoverratio,effectivefieldgoalpercentage,trueshootingpercentage,usagepercentage,estimatedusagepercentage,estimatedpace,pace,paceper40,possessions,pie
0,0022301199,1610612745,Houston,Rockets,HOU,rockets,1641708,Amen,Thompson,A. Thompson,...,13.0,0.889,0.872,0.124,0.124,105.18,105.18,87.65,87.0,0.174
1,0022301199,1610612745,Houston,Rockets,HOU,rockets,1631095,Jabari,Smith Jr.,J. Smith Jr.,...,0.0,0.368,0.377,0.225,0.225,103.64,103.64,86.37,71.0,0.074
2,0022301199,1610612745,Houston,Rockets,HOU,rockets,1629111,Jock,Landale,J. Landale,...,30.8,0.357,0.357,0.200,0.200,103.18,103.18,85.98,45.0,0.016
3,0022301199,1610612745,Houston,Rockets,HOU,rockets,1630224,Jalen,Green,J. Green,...,5.6,0.464,0.464,0.246,0.246,101.65,101.65,84.71,48.0,0.122
4,0022301199,1610612745,Houston,Rockets,HOU,rockets,1641715,Cam,Whitmore,C. Whitmore,...,4.2,0.588,0.573,0.194,0.194,107.38,107.38,89.49,80.0,0.115



🎯 STANDARDIZATION SUCCESS!
   ✅ All master tables use consistent lowercase column names
   ✅ Key fields: gameid, playerid, teamid (no underscores)
   ✅ Both V2 and V3 endpoints now use same naming convention


In [ ]:
games_df

,gameid,teamid,teamcity,teamname,teamtricode,teamslug,personid,firstname,familyname,namei,...,turnoverratio,effectivefieldgoalpercentage,trueshootingpercentage,usagepercentage,estimatedusagepercentage,estimatedpace,pace,paceper40,possessions,pie
0,0022301199,1610612745,Houston,Rockets,HOU,rockets,1641708,Amen,Thompson,A. Thompson,...,13.0,0.889,0.872,0.124,0.124,105.18,105.18,87.65,87.0,0.174
1,0022301199,1610612745,Houston,Rockets,HOU,rockets,1631095,Jabari,Smith Jr.,J. Smith Jr.,...,0.0,0.368,0.377,0.225,0.225,103.64,103.64,86.37,71.0,0.074
2,0022301199,1610612745,Houston,Rockets,HOU,rockets,1629111,Jock,Landale,J. Landale,...,30.8,0.357,0.357,0.200,0.200,103.18,103.18,85.98,45.0,0.016
3,0022301199,1610612745,Houston,Rockets,HOU,rockets,1630224,Jalen,Green,J. Green,...,5.6,0.464,0.464,0.246,0.246,101.65,101.65,84.71,48.0,0.122
4,0022301199,1610612745,Houston,Rockets,HOU,rockets,1641715,Cam,Whitmore,C. Whitmore,...,4.2,0.588,0.573,0.194,0.194,107.38,107.38,89.49,80.0,0.115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27753,0022100784,1610612761,Toronto,Raptors,TOR,raptors,1630551,Justin,Champagnie,J. Champagnie,...,100.0,0.000,0.000,0.111,0.111,92.34,92.34,76.95,9.0,-0.057
27754,0022100784,1610612761,Toronto,Raptors,TOR,raptors,1630625,Dalano,Banton,D. Banton,...,0.0,0.333,0.333,0.158,0.158,87.46,87.46,72.89,14.0,0.140
27755,0022100784,1610612761,Toronto,Raptors,TOR,raptors,1630201,Malachi,Flynn,M. Flynn,...,0.0,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.0,0.000
27756,0022100784,1610612761,Toronto,Raptors,TOR,raptors,1629004,Svi,Mykhailiuk,S. Mykhailiuk,...,0.0,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.0,0.000


: 